<a href="https://colab.research.google.com/github/tarannummuzaffar/super/blob/main/Image%20classification_Assignment_3_Week_5_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Welcome to Assignment 3 on Deep Learning for Computer Vision.**

This assignment is based on the content you learned in Week-5 of course.


#### **Instructions**
1. Use Python 3.x to run this notebook
2. Write your code only in between the lines 'YOUR CODE STARTS HERE' and 'YOUR CODE ENDS HERE'.
you should not change anything else in the code cells, if you do, the answers you are supposed to get at the end of this assignment might be wrong.
3. Read documentation of each function carefully.
4. All the Best!

##MNIST classification using CNN

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F
import timeit
import unittest

## Please DONOT remove these lines.
torch.manual_seed(2024)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(2024)

### Data Loading and Pre-processing

In [ ]:
# check availability of GPU and set the device accordingly
#### YOUR CODE STARTS HERE ####
device = ___
#### YOUR CODE ENDS HERE ####

# Hyper parameters
num_epochs = 10
num_classes = 10
learning_rate = 0.001

# define a transforms for preparing the dataset
# for normalization of the MNIST dataset, take mean=0.1307 and std=0.3081

#### YOUR CODE STARTS HERE ####
transform = ___
#### YOUR CODE ENDS HERE ####

In [ ]:
# Load the MNIST training, test datasets using `torchvision.datasets.MNIST` using the transform defined above
#### YOUR CODE STARTS HERE ####
train_dataset = ___
test_dataset = ___
#### YOUR CODE ENDS HERE ####

In [ ]:
# create dataloaders for training and test datasets
# use a batch size of 32 and set shuffle=True for the training set
#### YOUR CODE STARTS HERE ####
train_dataloader = ___
test_dataloader = ___
#### YOUR CODE ENDS HERE ####


### Network Definition

In [ ]:
# Convolutional neural network (Two convolutional layers)
class ConvolutionNet(nn.Module):
    def __init__(self, num_classes=10):
        super( ConvolutionNet, self).__init__()

        # define a sequential module with
        # 1. conv layer with input channel as 1, output channels as 32, kernel size of 5, stride of 1 and padding 2
        # 2. 2D BatchNorm of 32 features
        # 3. ReLU activation
        # 4. 2D MaxPool with kernel size of 2 and stride of 2

        #### YOUR CODE STARTS HERE ####
        self.conv_block1 = ___
        #### YOUR CODE ENDS HERE ####

        # define a sequential module with
        # 1. conv layer with input channel as 32, output channels as 16, kernel size of 7, stride of 1 and padding 3
        # 2. 2D BatchNorm of 16 features
        # 3. ReLU activation
        # 4. 2D MaxPool with kernel size of 2 and stride of 2

        #### YOUR CODE STARTS HERE ####
        self.conv_block2 =___
        #### YOUR CODE ENDS HERE ####

        # define a linear(dense) layer with output features corresponding to the number of classes in the dataset

        #### YOUR CODE STARTS HERE ####
        self.fc = ___
        #### YOUR CODE ENDS HERE ####

    def forward(self, x):
        # Use the sequential convolution blocks defined above (conv_block1--> conv_block2-->fc) and
        # write the forward pass.

        #### YOUR CODE STARTS HERE ####
        output = ___
        output = ___

        # Reshape appropiately
        output = ___

        output = ___
        #### YOUR CODE ENDS HERE ####
        return output


In [ ]:
model = ConvolutionNet(num_classes).to(device)
model.conv_block2[0].weight.shape

### Question 1

What is the size of parameter matrix corresponding to convolution layer of second sequential block ?

1. 32x16x5x5
2. 32x32x6x6
3. 16x32x7x7
4. 32x16x4x4


### Training and Inference

In [ ]:
#define the model
#### YOUR CODE STARTS HERE ####
model = ___
#### YOUR CODE ENDS HERE ####


#define cross entropy loss and Adam optimizer using learning rate=learning_rate
#### YOUR CODE STARTS HERE ####
# Loss and optimizer
criterion = ___
optimizer = ___
#### YOUR CODE ENDS HERE ####

# Train the model
total_step = len(train_dataloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dataloader):
       #### YOUR CODE STARTS HERE ####
        # send the image, target to the device
        images = ___
        labels = ___

        # flush out the gradients stored in optimizer
        ___

        # pass the image to the model and assign the output to variable named output
        output = ___

        # calculate the loss (use cross entropy in pytorch)
        loss = ____

        # do a backward pass
        ____

        # update the weights
        ____
       #### YOUR CODE ENDS HERE ####
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
      ### YOUR CODE STARTS HERE ####
        # send the image, target to the device
        images = ___
        labels = ___
        # pass the image to the model and assign the output to variable named output
        outputs = ___
      #### YOUR CODE ENDS HERE ####
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model : {} %'.format(100 * correct / total))

#### Question-2

Report the final test accuracy displayed above (If you are not getting the exact number shown in options, please report the closest number).
1. 84%
2. 76%
3. 99%
4. 57%


#### Question-3

Report the loss value at the step 1800? (If you are not getting the exact number shown in options, please report the closest number).
1. 0.0042
2. 0.15
3. 0.00065
4. 1.4


In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Create a Function to get embeddings
def get_embeddings(model, dataloader, device):
    # YOUR CODE STARTS HERE
    model.eval()
    embeddings = []
    labels = []
    with torch.no_grad():
        for images, batch_labels in dataloader:
            #Send images to GPU
            images = ___
            # Get the output of the second-to-last layer (before the final fully connected layer)
            embedding = ___
            # Flatten the embedding
            embedding = ___
            # Append the embedding and labels to the lists
            ___

    #YOUR CODE ENDS HERE
    return np.vstack(embeddings), np.concatenate(labels)

# Get embeddings for the test set
test_embeddings, test_labels = get_embeddings(model, test_dataloader, device)

# Apply t-SNE

tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(test_embeddings)

# Visualize the results
plt.figure(figsize=(10, 8))
scatter = plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c=test_labels, cmap='tab10')
plt.colorbar(scatter)
plt.title('t-SNE visualization of MNIST test set')
plt.xlabel('t-SNE feature 1')
plt.ylabel('t-SNE feature 2')
plt.show()

#### Question-4

Embedding of which class/digit is closest to the embedding of 7?
1. 3
2. 5
3. 8
4. 1


#### Question-5

Embedding of which class/digit is farthest to the embedding of 4?
1. 1
2. 6
3. 9
4. 3


In [ ]:
import matplotlib.pyplot as plt

# Get the weights of the first convolutional layer
#YOUR CODE STARTS HERE
filters = ___
#YOUR CODE ENDS HERE

print(f"shape of the filters is: {filters.shape}")

# Normalize the filters for better visualization
min_val = filters.min()
max_val = filters.max()
filters = (filters - min_val) / (max_val - min_val)

# Plot the filters
fig, axes = plt.subplots(4, 8, figsize=(12, 6))

for i, ax in enumerate(axes.flat):
    if i < filters.shape[0]:
        # Each filter is in the shape of (out_channels, in_channels, height, width)
        # We take the first in_channel (0) since it's grayscale, so only one channel exists
        ax.imshow(filters[i, 0, :, :], cmap='gray')
        ax.axis('off')
    else:
        ax.axis('off')

plt.show()


#### Question-6

What is the shape of filters?
1. (32,16 , 5, 5)
2. (32, 1, 5, 5)
3. (16, 32, 5, 5)
4. (16, 1)
